# Experiment 2 - Relations

Now that we have our model capable of detecting components, let us move on to estimating the number of relations. Initially, we considered splitting each abstract into sentences and labeling all possible combinations of sentences as either `No related` or `Related`, corresponding to 0 and 1, respectively. However, this led to an imbalanced and overly difficult task, as the majority of sentence pairs were unrelated. 

To address this, we approached the problem differently. Instead of trying to predict relationships between all possible sentence pairs, we reformulated the task to focus on the components directly. Specifically, we label each component as either *In a relationship* or *Not in a relationship*. A component is considered *In a relationship* if it serves as the first member (*source*) of at least one relation, and *Not in a relationship* otherwise. 
Consider the following annotation:

    R1	Partial-Attack Arg1:T10 Arg2:T11	
    R2	Support Arg1:T8 Arg2:T9	
    R3	Support Arg1:T7 Arg2:T10	
    R4	Support Arg1:T6 Arg2:T10	
    R5	Support Arg1:T5 Arg2:T9	
    R6	Support Arg1:T2 Arg2:T4	
    R7	Partial-Attack Arg1:T3 Arg2:T4	
    R8	Partial-Attack Arg1:T1 Arg2:T4	

In this annotation, there are eight relations, corresponding to the `R1` through `R8` entries. Each relation has a source, marked as `Arg1`, and a target, marked as `Arg2`. For example, in `R1`, `T10` is the source, and `T11` is the target. Using this information, we label components based on whether they appear as `Arg1` in any relation. Components like `T1`, `T2`, `T3`, `T5`, `T6`, `T7`, `T8`, and `T10` are labeled as `Relationship` (1), while components like `T4`, `T9`, and `T11` are labeled as `Not relationship` (0), as well as non-components. This reformulation simplifies the task from predicting relationships between pairs of sentences to predicting whether a given component participates in an outgoing relationship.

In [ ]:
from experiment_2_code import *

C:\Users\Antonio\anaconda3\envs\NLP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Antonio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Antonio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### 1. Preprocessing
Here we preprocess our data by splitting the abstracts into sentences. Each combination of two sentences is then labeled as either being a `Relationship`, labeled as `1`, or `Not relationship`, labeled as `0`.

In [10]:
train_set = read_brat_dataset('../data/train/neoplasm_train') 
val_set = read_brat_dataset('../data/dev/neoplasm_dev')

glaucoma_test = read_brat_dataset('../data/test/glaucoma_test')
neoplasm_test = read_brat_dataset('../data/test/neoplasm_test')
mixed_test = read_brat_dataset('../data/test/mixed_test')

test_set = glaucoma_test + neoplasm_test + mixed_test

It happens 24 times that a component is already in a relation in neoplasm_train.
It happens 4 times that a component is already in a relation in neoplasm_dev.
It happens 21 times that a component is already in a relation in glaucoma_test.
It happens 5 times that a component is already in a relation in neoplasm_test.
It happens 9 times that a component is already in a relation in mixed_test.


In [11]:
label_counts_train, avg_sentences_per_file_train = compute_dataset_statistics(train_set, dataset_name="train")
label_counts_test, avg_sentences_per_file_test = compute_dataset_statistics(test_set, dataset_name="test")

- Train set:
	Label 0: 3251 samples
	Label 1: 1394 samples

	There are 2 different labels in the train set -> [0, 1]
	Average number of sentences per file in train set: 13
	Max sentence length: 107
	Average relationships per file: 4.06
	Average no relationships per file: 9.29

- Test set:
	Label 0: 2743 samples
	Label 1: 1085 samples

	There are 2 different labels in the test set -> [0, 1]
	Average number of sentences per file in test set: 14
	Max sentence length: 91
	Average relationships per file: 4.09
	Average no relationships per file: 10.20



In [12]:
train_collection = FilenameLabelledCollection([data['sentence'] for data in train_set], 
                                                 [data['label'] for data in train_set], 
                                                 [data['filename'] for data in train_set])

# train_collection, val_collection = train_collection.split_stratified(0.7)
# train_collection, val_collection = train_collection.split_stratified_by_filenames(0.75)
val_collection = FilenameLabelledCollection([data['sentence'] for data in val_set], 
                                                 [data['label'] for data in val_set], 
                                                 [data['filename'] for data in val_set])

test_collection = FilenameLabelledCollection([data['sentence'] for data in test_set], 
                                                 [data['label'] for data in test_set], 
                                                 [data['filename'] for data in test_set])

glaucoma_collection = FilenameLabelledCollection([data['sentence'] for data in glaucoma_test], 
                                                 [data['label'] for data in glaucoma_test], 
                                                 [data['filename'] for data in glaucoma_test])

neoplasm_collection = FilenameLabelledCollection([data['sentence'] for data in neoplasm_test], 
                                                 [data['label'] for data in neoplasm_test], 
                                                 [data['filename'] for data in neoplasm_test])

mixed_collection = FilenameLabelledCollection([data['sentence'] for data in mixed_test], 
                                                 [data['label'] for data in mixed_test], 
                                                 [data['filename'] for data in mixed_test])

In [13]:
indexer = qp.data.preprocessing.IndexTransformer(min_df=1)

# Create and index the dataset
abs_dataset = CustomDataset(training=train_collection, test=test_collection, val=val_collection)
index(abs_dataset, indexer, inplace=True)

# Index the test collections
index(glaucoma_collection, indexer, fit=False, inplace=True)
index(neoplasm_collection, indexer, fit=False, inplace=True)
index(mixed_collection, indexer, fit=False, inplace=True)

qp.environ['SAMPLE_SIZE'] = avg_sentences_per_file_train

indexing: 100%|███████████████████████████████████████████████| 1208/1208 [00:00<00:00, 75498.72it/s]


### 2. Classifier

In [7]:
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective(trial, abs_dataset), n_trials=50)

print("Best trial:")
trial = study.best_trial
print(f"  Macro F1 Score: {trial.value}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-10-30 16:26:42,953] A new study created in memory with name: no-name-e3381943-a2e9-44f2-bd9f-bc1bdcb97ef5


Starting trial 0 with parameters:
    Embedding size: 172 - Hidden size: 281
    Optimizer: Adam (lr: 0.0008485856927597929) - Scheduler: CosineAnnealingWarmRestarts (params: {'T_0': 12, 'T_mult': 5})
[NeuralNetwork running on cpu]


[CNNnet] training epoch=7 tr-loss=0.14423 tr-acc=94.66% tr-macroF1=93.62% patience=1/5 val-loss=0.530C:\Users\Antonio\Documents\UniBO\NLP\project\nlp_unibo_2023-24_project\experiments_antonio\experiment_4_code.py:672: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded f

Training ended by patience exhausted; loading best model parameters from ../checkpoint/relations/classifier_net.dat from epoch 2
Performing a final training pass over the validation set...


[I 2024-10-30 16:29:40,136] Trial 0 finished with value: 0.8037904523511532 and parameters: {'embedding_size': 172, 'hidden_size': 281, 'optimizer': 'Adam', 'lr': 0.0008485856927597929, 'scheduler': 'CosineAnnealingWarmRestarts', 'T_0': 12, 'T_mult': 5}. Best is trial 0 with value: 0.8037904523511532.


...

[Training complete] - Best loss on validation set: 0.40929220616817474 - Best f1 on validation set: 0.786531279178338
Starting trial 49 with parameters:
    Embedding size: 195 - Hidden size: 278
    Optimizer: Adam (lr: 0.000523688821899043) - Scheduler: CosineAnnealingLR (params: {'T_max': 9})
[NeuralNetwork running on cpu]


[CNNnet] training epoch=8 tr-loss=0.05311 tr-acc=98.11% tr-macroF1=97.74% patience=1/5 val-loss=0.487C:\Users\Antonio\Documents\UniBO\NLP\project\nlp_unibo_2023-24_project\experiments_antonio\experiment_4_code.py:672: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded f

Training ended by patience exhausted; loading best model parameters from ../checkpoint/relations/classifier_net.dat from epoch 3
Performing a final training pass over the validation set...


[I 2024-10-30 20:43:29,176] Trial 49 finished with value: 0.7943555065857224 and parameters: {'embedding_size': 195, 'hidden_size': 278, 'optimizer': 'Adam', 'lr': 0.000523688821899043, 'scheduler': 'CosineAnnealingLR', 'T_max': 9}. Best is trial 47 with value: 0.8110541940329175.


[Training complete] - Best loss on validation set: 0.3936632573604584 - Best f1 on validation set: 0.7943555065857224
Best trial:
  Macro F1 Score: 0.8110541940329175
  Params:
    embedding_size: 195
    hidden_size: 278
    optimizer: Adam
    lr: 0.0005161449102180434
    scheduler: CosineAnnealingLR
    T_max: 11


We can now train and test using the obtained hyperparameters.

In [14]:
set_seed(42)

embedding_size = 195
hidden_size = 278   
lr = 0.0005161449102180434

cnn_module = CNNnet(
    abs_dataset.vocabulary_size,
    abs_dataset.training.n_classes,
    embedding_size=embedding_size,
    hidden_size=hidden_size
)

optimizer = Adam(cnn_module.parameters(), lr=lr)
scheduler = CosineAnnealingLR(optimizer, T_max=11)

cnn_classifier = ScheduledNeuralClassifierTrainer(
    cnn_module,
    lr_scheduler=scheduler,
    optim = optimizer,
    device='cpu',
    checkpointpath='../checkpoints/relations/classifier_net.dat',
    padding_length=107,
    patience=10
)

cnn_classifier.fit(*abs_dataset.training.Xy, *abs_dataset.val.Xy)

[NeuralNetwork running on cpu]


[CNNnet] training epoch=19 tr-loss=0.02235 tr-acc=99.31% tr-macroF1=99.18% patience=1/10 val-loss=0.6C:\Users\Antonio\Documents\UniBO\NLP\project\nlp_unibo_2023-24_project\experiments_antonio\experiment_4_code.py:671: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded f

Training ended by patience exhausted; loading best model parameters from ../checkpoints/relations/classifier_net.dat from epoch 9
Performing a final training pass over the validation set...
[Training complete] - Best loss on validation set: 0.47745291888713837 - Best f1 on validation set: 0.8110541940329175


In [15]:
f1_train = 1-qp.error.f1e(abs_dataset.training.labels, cnn_classifier.predict(abs_dataset.training.instances))
accuracy_train = 1-qp.error.acce(abs_dataset.training.labels, cnn_classifier.predict(abs_dataset.training.instances))
print('- Train set:')
print(f'\tF1: {f1_train}')    
print(f'\tAccuracy: {accuracy_train}')    

f1_test_glaucoma = 1-qp.error.f1e(glaucoma_collection.labels, cnn_classifier.predict(glaucoma_collection.instances))
accuracy_test_glaucoma = 1-qp.error.acce(glaucoma_collection.labels, cnn_classifier.predict(glaucoma_collection.instances))

print('- Glaucoma test set:')
print(f'\tF1: {f1_test_glaucoma}')    
print(f'\tAccuracy: {accuracy_test_glaucoma}')    

f1_test_neoplasm = 1-qp.error.f1e(neoplasm_collection.labels, cnn_classifier.predict(neoplasm_collection.instances))
accuracy_test_neoplasm = 1-qp.error.acce(neoplasm_collection.labels, cnn_classifier.predict(neoplasm_collection.instances))

print('- Neoplasm test set:')
print(f'\tF1: {f1_test_neoplasm}')    
print(f'\tAccuracy: {accuracy_test_neoplasm}')

f1_test_mixed = 1-qp.error.f1e(mixed_collection.labels, cnn_classifier.predict(mixed_collection.instances))
accuracy_test_mixed = 1-qp.error.acce(mixed_collection.labels, cnn_classifier.predict(mixed_collection.instances))

print('- Mixed test set:')
print(f'\tF1: {f1_test_mixed}')    
print(f'\tAccuracy: {accuracy_test_mixed}')

- Train set:
	F1: 0.9992308085066453
	Accuracy: 0.9993541442411195
- Glaucoma test set:
	F1: 0.731208145908966
	Accuracy: 0.764751552795031
- Neoplasm test set:
	F1: 0.7878662886196062
	Accuracy: 0.8205705705705706
- Mixed test set:
	F1: 0.7414842889520707
	Accuracy: 0.7855960264900662


In [16]:
infer(test_set, indexer, comp_quantifier=None, comp_classifier=None, rel_quantifier=None, rel_classifier=cnn_classifier, filename=None, use_tokenizer=True)

File 9643663 - Relations: [0] - Text:
 To evaluate the efficacy and tolerability of 'Casodex' monotherapy (150 mg daily) for metastatic and locally advanced prostate cancer. A total of 1,453 patients with either confirmed metastatic disease (M1), or T3/T4 non-metastatic disease with elevated prostate-specific antigen (M0) were recruited into one of two identical, multicentre, randomised studies to compare 'Casodex' 150 mg/day with castration. The protocols allowed for combined analysis. At a median follow-up period of approximately 100 weeks for both studies, 'Casodex' 150 mg was found to be less effective than castration in patients with metastatic disease (M1) at entry (hazard ratio of 1.30 for time to death) with a difference in median survival of 6 weeks. In symptomatic M1 patients, 'Casodex' was associated with a statistically significant improvement in subjective response (70%) compared with castration (58%). Analysis of a validated quality-of-life questionnaire proved an advanta

indexing: 100%|███████████████████████████████████████████████████| 13/13 [00:00<00:00, 13004.04it/s]

	Classification:
		# Ground truth relations: (0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0)
		# Predicted relations labels: [0 0 0 0 1 0 1 0 0 0 0 0 0]


We obtain worse results if compared with the components model: this is expected, as the number of positive labels is now a subgroup of the positive labels used to train the previous model, leading to an imbalanced dataset and a more difficult task.

### 3. QuaNet 
We will include the description of `QuaNet` again, as it may prove useful. `QuaNet` observes the classification predictions to learn higher-order *quantification embeddings*, which are then refined by incorporating quantification predictions of simple classify-and-count-like methods.

![architecture](./images/quanet_architecture.png)

The QuaNet architecture (see Figure 1) consists of two main components: a **recurrent component** and a **fully connected component**.

#### 3.1 Recurrent Component: Bidirectional LSTM
- The core of the model is a **Bidirectional LSTM** (Long Short-Term Memory), a type of recurrent neural network. 
- The LSTM receives as input a **list of pairs** $⟨Pr(c|x), \vec{x}⟩$, where:
  - $Pr(c|x)$ is the probability that a classifier $h$ assigns class $c$ to document $x$.
  - $\vec{x}$ is the **document embedding**, a vector representing the document's content.
- The list is **sorted by the value of $Pr(c|x)$**, meaning the documents are arranged from least to most likely to belong to class $c$.
  
The **intuition** behind this approach is that the LSTM will "learn to count" positive and negative examples. By observing the ordered sequence of probabilities, the LSTM should learn to recognize the point where the documents switch from negative to positive examples. The document embedding $\vec{x}$ helps the LSTM assign different importance to each document when making its prediction.

The output of the LSTM is called a **quantification embedding**—a dense vector representing the information about the quantification task learned from the input data.

#### 3.2 Fully Connected Component
- The vector returned by the LSTM is combined with additional information, specifically **quantification-related statistics**:
  - $\hat{p}_c^{CC}(D)$, $\hat{p}_c^{ACC}(D)$, $\hat{p}_c^{PCC}(D)$, and $\hat{p}_c^{PACC}(D)$, which are quantification predictions from different methods.
  - $tpr_b$, $fpr_b$, $tpr_s$, and $fpr_s$, aggregate statistics related to true positive and false positive rates, which are easy to compute from the classifier $h$ using a validation set.

This combined vector then passes through the second part of the architecture, which is made up of **fully connected layers** with **ReLU activations**. These layers adjust the quantification embedding using the additional statistics from the classifier to improve the accuracy of the quantification.

The final output is a prediction $\hat{p}_c^{QuaNet}(c|D)$, which represents the probability of class $c$ for the dataset $D$, produced by a **softmax layer**.

QuaNet could use quantification predictions from many methods, but it focuses on those that are **computationally efficient** (like CC, ACC, PCC, and PACC). This ensures that the process remains fast while still providing sufficient information for accurate predictions.

### Details

| Layer | Type | Dimensions | Activation | Dropout |
|---|---|---|---|---|
| Input | LSTM | 128 | N/A | N/A |
| Dense 1 | Dense | 1024 | ReLU | 0.5 |
| Dense 2 | Dense | 512 | ReLU | 0.5 |
| Output | Dense | 2 | Softmax | N/A |

- The LSTM has **64 hidden dimensions**, and since it’s bidirectional, the final LSTM output has **128 dimensions**.
- This LSTM output is concatenated with the **8 quantification statistics** (giving a total of 136 dimensions), which is then fed into:
  - **Two dense layers** with **1,024** and **512 dimensions**, each using **ReLU activation** and **0.5 dropout**.
  - Finally, the output is passed through a **softmax layer** of size 2 to make the final class prediction.


In [17]:
# train QuaNet (alternatively, we can set fit_classifier=True and let QuaNet train the classifier)
set_seed(42)

quantifier = QuaNet(cnn_classifier, qp.environ['SAMPLE_SIZE'], qdrop_p=0, device='cpu', checkpointdir='../checkpoints/relations', checkpointname='Quanet-Relations')
quantifier.fit(abs_dataset.training, fit_classifier=False)

QuaNetModule(
  (lstm): LSTM(102, 64, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0, inplace=False)
  (ff_layers): ModuleList(
    (0): Linear(in_features=136, out_features=1024, bias=True)
    (1): Linear(in_features=1024, out_features=512, bias=True)
  )
  (output): Linear(in_features=512, out_features=2, bias=True)
)


  0%|                                                                        | 0/500 [00:00<?, ?it/s]C:\Users\Antonio\anaconda3\envs\NLP\Lib\site-packages\quapy\method\_neural.py:233: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  ptrue = torch.as_tensor([sample_data.prevalence()], dtype=torch.float, device=self.device)
[QuaNet] epoch=1 [it=499/500]	tr-mseloss=0.00379 tr-maeloss=0.03395	val-mseloss=-1.00000 val-maeloss=
100%|██████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 48.47it/s]
[QuaNet] epoch=2 [it=499/500]	tr-mseloss=0.00036 tr-maeloss=0.01412	val-mseloss=0.00048 val-maeloss=0
100%|██████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 49.54it

training ended by patience exhausted; loading best model parameters in ../checkpoints/relations\Quanet-Relations for epoch 29



C:\Users\Antonio\anaconda3\envs\NLP\Lib\site-packages\quapy\method\_neural.py:188: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.quanet.load_state_dict(torch.load(chec

QuaNetTrainer(classifier__batch_size=64, classifier__batch_size_test=512,
              classifier__device=device(type='cpu'), classifier__drop_p=0.5,
              classifier__embedding_size=195, classifier__epochs=200,
              classifier__hidden_size=278, classifier__kernel_heights=[3, 5, 7],
              classifier__lr=0.001, classifier__padding_length=107,
              classifier__patience=10, classifier__repr_size=100,
              classifier__stride=1, classifier__weight_decay=0, qdrop_p=0)

In [26]:
print('- Train set:')
result_train = evaluate(collection=abs_dataset.training, n=[1,3,5,10, qp.environ['SAMPLE_SIZE'], 15], quantifier=quantifier)

print('- Val set:')
result_train = evaluate(collection=abs_dataset.val, n=[1,3,5,10, qp.environ['SAMPLE_SIZE'], 15], quantifier=quantifier)

print('- Glaucoma test set:')
result_test = evaluate(collection=glaucoma_collection, n=[1,3,5,10, qp.environ['SAMPLE_SIZE'], 15], quantifier=quantifier)

print('- Neoplasm test set:')
result_test = evaluate(collection=neoplasm_collection, n=[1,3,5,10, qp.environ['SAMPLE_SIZE'], 15], quantifier=quantifier)

print('- Mixed test set:')
result_test = evaluate(collection=mixed_collection, n=[1,3,5,10, qp.environ['SAMPLE_SIZE'], 15], quantifier=quantifier)

- Train set:
Error Metric	Standard	ByDoc (n=1)	ByDoc (n=3)	ByDoc (n=5)	ByDoc (n=10)	ByDoc (n=13)	ByDoc (n=15)
------------	--------	-----------	-----------	-----------	------------	------------	------------
AE             	0.0004         	0.0026         	0.0021         	0.0019         	0.0016         	0.0014         	0.0018         
RAE            	0.0009         	0.0069         	0.0047         	0.0041         	0.0034         	0.0031         	0.0039         
MSE            	0.0000         	0.0000         	0.0000         	0.0000         	0.0000         	0.0000         	0.0000         
MAE            	0.0004         	0.0026         	0.0021         	0.0019         	0.0016         	0.0014         	0.0018         
MRAE           	0.0009         	0.0069         	0.0047         	0.0041         	0.0034         	0.0031         	0.0039         
MKLD           	0.0000         	0.0001         	0.0000         	0.0000         	0.0000         	0.0000         	0.0000         
- Val set:
Error Metric	S

As expected, results are once again slightly worse, reflecting the increased difficulty of the task.

In [ ]:
infer(test_set, indexer, comp_quantifier=None, comp_classifier=None, rel_quantifier=quantifier, rel_classifier=cnn_classifier, filename=None, use_tokenizer=True)